# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [56]:
create_table1 = 'CREATE TABLE IF NOT EXISTS session_info'
create_table1 = create_table1 + "(sessionId int,\
                                  itemInSession int,\
                                  artist text,\
                                  song text,\
                                  length float,\
                                  PRIMARY KEY(sessionId, itemInSession)\
                                  )"

try:
    session.execute(create_table1)
except Exception as e:
    print(e)

In [57]:
create_table2 = 'CREATE TABLE IF NOT EXISTS user_info'
create_table2 = create_table2 + "(sessionId int,\
                                  itemInSession int,\
                                  userId int,\
                                  firstName text,\
                                  lastName text,\
                                  artist text,\
                                  song text,\
                                  PRIMARY KEY (userId, sessionId, itemInSession)\
                                  )WITH CLUSTERING ORDER BY(sessionId ASC, itemInSession ASC)"
try:
    session.execute(create_table2)
except Exception as e:
    print(e)

In [58]:
create_table3 = 'CREATE TABLE IF NOT EXISTS song_info'
create_table3 = create_table3 + "(firstName text,\
                                  lastName text,\
                                  song text,\
                                  userId int,\
                                  PRIMARY KEY (song, userId)\
                                  )"
try:
    session.execute(create_table3)
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [59]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Table1
        query = "INSERT INTO session_info (sessionId,\
                                           itemInSession,\
                                           artist,\
                                           song,\
                                           length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
        #Table2
        query = "INSERT INTO user_info (sessionId,\
                                        itemInSession,\
                                        userId,\
                                        firstName,\
                                        lastName,\
                                        artist,\
                                        song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[1], line[4], line[0], line[9]))
        
        #Table3
        query = "INSERT INTO song_info (firstName,\
                                        lastName,\
                                        song,\
                                        userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [60]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
table1 = "SELECT * from session_info"
df1 = pd.DataFrame([*session.execute(table1)])

table2 = "SELECT * from user_info"
df2 = pd.DataFrame([*session.execute(table2)])

table3 = "SELECT * from song_info"
df3 = pd.DataFrame([*session.execute(table3)])

In [61]:
df1.head()

,sessionid,iteminsession,artist,length,song
0,23,0,Regina Spektor,191.085266,The Calculation (Album Version)
1,23,1,Octopus Project,250.957916,All Of The Champs That Ever Lived
2,23,2,Tegan And Sara,180.061584,So Jealous
3,23,3,Dragonette,153.390564,Okay Dolores
4,23,4,Lil Wayne / Eminem,229.589752,Drop The World


In [62]:
df2.head()

,userid,sessionid,iteminsession,artist,firstname,lastname,song
0,23,177,2,Dwight Yoakam,Morris,Gilmore,You're The One
1,23,351,0,Foals,Morris,Gilmore,Blue Blood
2,23,351,1,'N Sync/Phil Collins,Morris,Gilmore,Trashin' The Camp (Phil And 'N Sync Version)
3,23,841,0,Eminem,Morris,Gilmore,Just Lose It
4,53,52,2,Mynt,Celeste,Williams,Playa Haters


In [63]:
df3.head()

,song,userid,firstname,lastname
0,Wonder What's Next,49,Chloe,Cuevas
1,In The Dragon's Den,49,Chloe,Cuevas
2,Too Tough (1994 Digital Remaster),44,Aleena,Kirby
3,Rio De Janeiro Blue (Album Version),49,Chloe,Cuevas
4,My Place,15,Lily,Koch


## Execute query 1

1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [65]:
query1="""
SELECT artist, song, length 
    FROM session_info
    WHERE sessionId=338 
    AND itemInSession=4
"""                 
df_q1 = pd.DataFrame(list(session.execute(query1)))

In [66]:
df_q1

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Execute query 2

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [72]:
query2="""
SELECT artist, song, firstName, lastName
    FROM user_info
    WHERE userId=10 
    AND sessionId=182
"""                 
df_q2 = pd.DataFrame(list(session.execute(query2)))

In [73]:
df_q2

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Execute query 3

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [77]:
query3="""
SELECT firstName, lastName
    FROM song_info
    WHERE song='All Hands Against His Own' 
"""                 
df_q3 = pd.DataFrame(list(session.execute(query3)))

In [78]:
df_q3

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [53]:
query = "DROP TABLE IF EXISTS session_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [54]:
query = "DROP TABLE IF EXISTS user_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [55]:
query = "DROP TABLE IF EXISTS song_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()